# 16차시 분류: 로지스틱 회귀분석(Logistic Regression)

## 01 로지스틱 회귀분석 개요

### 로지스틱 회귀분석 특징

- 이항 로지스틱 회귀 분석은 종속변수가 0과 1이며 베르누이 분포를 따를 경우 사용
- 모델의 산출 값은 각 데이터가 1이 될 확률이며 이진 분류를 위해서 경계값(threshold)이 필요
- 모델 평가를 위해 각종 분류 관련 지표 및 AUC 활용

### 승산비(OR, Odds Ratio)

- 특정 독립변수를 제외한 나머지 값을 고정하고 해당 독립변수가 1 증가 시 변화하는 승산(odds)의 비

## 02 주요 함수 및 메서드 소개

### statsmomdels - Logit()

- 로지스틱 회귀분석을 실시하는 statsmodels의 함수
- endog, exog 인자에 각각 종속변수와 독립변수를 할당
- 산출 모델 객체의 params 어트리뷰트에 모델의 계수 저장
- 산출 모델 객체의 predict() 메서드로 예측값을 생산하며 이는 종속변수가 1이 될 확률값

In [56]:
import pandas as pd
from statsmodels.api import Logit

In [17]:
df = pd.read_csv("강의자료/실습파일/iris.csv")
df.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
df["Species"].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [18]:
df["is_setosa"] = (df["Species"] == "setosa") + 0
df.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,is_setosa
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,1
3,4.6,3.1,1.5,0.2,setosa,1
4,5.0,3.6,1.4,0.2,setosa,1


In [5]:
model = Logit(endog = df["is_setosa"],
             exog = df.iloc[:, :2]).fit()
model

Optimization terminated successfully.
         Current function value: 0.036374
         Iterations 11


In [6]:
df.iloc[:, :2]

,Sepal.Length,Sepal.Width
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6
...,...,...
145,6.7,3.0
146,6.3,2.5
147,6.5,3.0
148,6.2,3.4


In [7]:
model.params

Sepal.Length    -7.529945
Sepal.Width     13.130734
dtype: float64

In [9]:
model.pvalues

Sepal.Length    0.000828
Sepal.Width     0.000989
dtype: float64

In [10]:
model.predict(df.iloc[:, :2])  # setosa일 확률

0      9.994767e-01
1      9.238236e-01
2      9.986784e-01
3      9.976885e-01
4      9.999337e-01
           ...     
145    1.575352e-05
146    4.509902e-07
147    7.102256e-05
148    1.149482e-01
149    6.467514e-03
Length: 150, dtype: float64

In [11]:
pred = model.predict(df.iloc[:, :2])
(pred > 0.5) + 0 

0      1
1      1
2      1
3      1
4      1
      ..
145    0
146    0
147    0
148    0
149    0
Length: 150, dtype: int32

### sklearn - LogisticRegression()

- 로지스틱 회귀분석을 실시하는 sklearn의 함수
- fit_intercept, solver 인자로 절편 적합 여부 및 최적화 알고리즘 설정 가능
- random_state 인자에 자연수를 할당하여 결과 고정 가능
- fit() 메서드에 독립변수 및 종속변수 할당
- 산출 모델 객체의 coef_ 어트리뷰트에 모델의 계수 저장
- 산출 모델 객체의 predict_proba() 메서드로 예측값을 생산하며 두 번째 열이 종속변수가 1이 될 확률값

In [12]:
from sklearn.linear_model import LogisticRegression

In [19]:
model = LogisticRegression(random_state = 123)
model.fit(X = df.iloc[:, :2],
          y = df["is_setosa"])
model

LogisticRegression(random_state=123)

In [20]:
model.coef_

array([[-3.38829757,  3.1645277 ]])

In [21]:
model.intercept_

array([8.32330389])

In [23]:
model.predict_proba(df.iloc[:5, :2])

array([[0.10727976, 0.89272024],
       [0.22895365, 0.77104635],
       [0.07413821, 0.92586179],
       [0.07261677, 0.92738323],
       [0.05873904, 0.94126096]])

In [24]:
pred = model.predict_proba(df.iloc[:5, :2])
pred = pred[:, 1]
pred

array([0.89272024, 0.77104635, 0.92586179, 0.92738323, 0.94126096])

In [25]:
(pred > 0.5) + 0

array([1, 1, 1, 1, 1])

### sklearn - roc_auc_score()

- AUC(Area Under Cureve)를 산출하는 sklearn의 함수
- y_true, y_score 인자에 각각 종속변수와 예측 확률값 할당

In [26]:
pred = model.predict_proba(df.iloc[:, :2])
pred = pred[:, 1]
pred[:10]

array([0.89272024, 0.77104635, 0.92586179, 0.92738323, 0.94126096,
       0.91436651, 0.97058885, 0.89484454, 0.93034007, 0.82210603])

In [27]:
from sklearn.metrics import roc_auc_score

In [28]:
roc_auc_score(y_true = df["is_setosa"],
             y_score = pred)

0.9999999999999999

### sklearn - accuracy_score()

- 분류모델의 정확도를 산출하는 sklearn의 함수
- y_pred와 y_true에 각각 예측 분류 결과와 실제 값을 할당

### sklearn - f1_score()

- 분류모델의 f1 값을 산출하는 sklearn의 함수
- y_pred와 y_true에 각각 예측 분류 결과와 실제 값을 할당

### sklearn - precision_score()

- 분류모델의 정밀도(precision)를 산출하는 sklearn의 함수
- y_pred와 y_true에 각각 예측 분류 결과와 실제 값을 할당

### sklearn - recall_score()

- 분류모델의 재현율(recall)를 산출하는 sklearn의 함수
- y_pred와 y_true에 각각 예측 분류 결과와 실제 값을 할당

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [31]:
accuracy_score(y_true = df["is_setosa"],
               y_pred = (pred > 0.8) + 0)

0.9466666666666667

In [32]:
f1_score(y_true = df["is_setosa"],
               y_pred = (pred > 0.8) + 0)

0.9130434782608696

In [33]:
precision_score(y_true = df["is_setosa"],
               y_pred = (pred > 0.8) + 0)

1.0

In [34]:
recall_score(y_true = df["is_setosa"],
               y_pred = (pred > 0.8) + 0)

0.84

## Q1 독립변수를 혈압, 혈당, BMI, 인슐린으로 하고 종속변수를 당뇨 여부로 할 경우 분류 정확도는 얼마인가?
1) diabetes.csv 파일 사용  
2) statsmodels 함수 사용  
3) 데이터는 학습:평가 = 8:2로 분리 후 계산  
4) Seed는 123

In [35]:
Q1 = pd.read_csv("강의자료/실습파일/diabetes.csv")
Q1.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [36]:
from sklearn.model_selection import train_test_split

In [38]:
Q1_train, Q1_test = train_test_split(Q1, train_size = 0.8, random_state = 123)
Q1_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
318,3,115,66,39,140,38.1,0.150,28,0
313,3,113,50,10,85,29.5,0.626,25,0
195,5,158,84,41,210,39.4,0.395,29,1
570,3,78,70,0,0,32.5,0.270,39,0
226,0,101,76,0,0,35.7,0.198,26,0


In [40]:
Q1_model = Logit(endog = Q1_train["Outcome"],
                 exog = Q1_train[["BloodPressure", "Glucose","BMI", "Insulin"]]).fit()
Q1_model

Optimization terminated successfully.
         Current function value: 0.626579
         Iterations 5


In [47]:
Q1_pred = Q1_model.predict(Q1_test[["BloodPressure", "Glucose","BMI", "Insulin"]])
Q1_pred

236    0.462956
395    0.507051
36     0.359735
210    0.314389
483    0.219876
         ...   
650    0.398274
579    0.561881
119    0.281386
593    0.386267
310    0.280089
Length: 154, dtype: float64

In [51]:
pred_class = (Q1_pred > 0.5) + 0
pred_class

236    0
395    1
36     0
210    0
483    0
      ..
650    0
579    1
119    0
593    0
310    0
Length: 154, dtype: int32

In [50]:
accuracy_score(y_true = Q1_test["Outcome"],
               y_pred = pred_class)

0.7012987012987013

## Q2 독립변수를 혈당, BMI, 나이로 하고 종속변수를 당뇨 여부로 할 경우 나이의 승산비는 얼마인가?
1) statsmodels 함수 사용

In [52]:
Q2 = pd.read_csv("강의자료/실습파일/diabetes.csv")
Q2.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [55]:
Q2_model = Logit(endog = Q2["Outcome"],
                 exog = Q2[["Glucose","BMI", "Age"]]).fit()
Q2_model.params

Optimization terminated successfully.
         Current function value: 0.656276
         Iterations 4


Glucose    0.009368
BMI       -0.035639
Age       -0.012898
dtype: float64

In [58]:
import numpy as np
np.exp(Q2_model.params)  # 승산비 odds ratio 계산

Glucose    1.009412
BMI        0.964989
Age        0.987184
dtype: float64

## Q3 독립변수를 혈당, BMI, 나이로 하고 종속변수를 당뇨 여부로 할 경우 모델의 AUC는 얼마인가?

In [59]:
Q3 = pd.read_csv("강의자료/실습파일/diabetes.csv")
Q3.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [62]:
Q3_model = Logit(endog = Q3["Outcome"],
                 exog = Q3[["Glucose","BMI", "Age"]]).fit()

Optimization terminated successfully.
         Current function value: 0.656276
         Iterations 4


In [68]:
Q3_pred = Q3_model.predict(Q3.loc[:, ["Glucose", "BMI", "Age"]])

In [69]:
roc_auc_score(y_true = Q3["Outcome"],
             y_score = Q3_pred)

0.5414253731343283